In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime

In [18]:
btc_usd = pd.read_csv("BTC-USD.csv")
btc_usd = btc_usd.loc[:,["Date","Close"]]
btc_cad = pd.read_csv("BTC-CAD.csv")
btc_cad = btc_cad.loc[:,["Date","Close"]]
btc_eur = pd.read_csv("BTC-EUR.csv")
btc_eur = btc_eur.loc[:,["Date","Close"]]
btc_gbp = pd.read_csv("BTC-GBP.csv")
btc_gbp = btc_gbp.loc[:,["Date","Close"]]
eur_cad = pd.read_csv("EUR-CAD.csv")
eur_cad = eur_cad.loc[:,["Date","Close"]]
eur_gbp = pd.read_csv("EUR-GBP.csv")
eur_gbp = eur_gbp.loc[:,["Date","Close"]]
eur_usd = pd.read_csv("EUR-USD.csv")
eur_usd = eur_usd.loc[:,["Date","Close"]]
gbp_cad = pd.read_csv("GBP-CAD.csv")
gbp_cad = gbp_cad.loc[:,["Date","Close"]]
gbp_usd = pd.read_csv("GBP-USD.csv")
gbp_usd = gbp_usd.loc[:,["Date","Close"]]
cad_usd = pd.read_csv("CAD-USD.csv")
cad_usd = cad_usd.loc[:,["Date","Close"]]

In [23]:
btc = btc_usd
btc.merge(btc_cad,right_on = "Date",left_on = "Date")
btc.merge(btc_eur,right_on = "Date",left_on = "Date")
btc.merge(btc_gbp,right_on = "Date",left_on = "Date")

,Date,Close_x,Close_y
0,2020-11-29,18177.484375,13635.293945
1,2020-11-30,19625.835938,14715.412109
2,2020-12-01,18802.998047,14010.114258
3,2020-12-02,19201.091797,14362.838867
4,2020-12-03,19445.398438,14455.398438
...,...,...,...
361,2021-11-25,57274.679688,43024.109375
362,2021-11-26,53569.765625,40161.789063
363,2021-11-27,54815.078125,41083.078125
364,2021-11-28,57248.457031,42913.328125
